In [64]:
import boto3
import pandas as pd
import numpy as np

### Download the following four CSV files from the `mindex-data-analytics-code-challenge` S3 bucket
This data represents the results of the Bengals 2021 NFL Season and the stats of the team’s top 3 receivers.
 - bengals.csv
 - boyd_receiving.csv
 - chase_receiving.csv
 - higgins_receiving.csv

In [28]:
s3 = boto3.resource('s3')
bucket_name = 'mindex-data-analytics-code-challenge'
bucket = s3.Bucket(bucket_name)
# TODO: make this work for Windows and Unix-like file systems
for obj in bucket.objects.all():
    s3.Object(bucket_name, obj.key).download_file(f'.\\{obj.key}')
    print(obj.key)

bengals.csv
boyd_receiving.csv
chase_receiving.csv
higgins_receiving.csv


### Use the pandas library to load each CSV into its own dataframe

In [33]:
bengals_df = pd.read_csv('bengals.csv')
boyd_df = pd.read_csv('boyd_receiving.csv')
chase_df = pd.read_csv('chase_receiving.csv')
higgins_df = pd.read_csv('higgins_receiving.csv')

### Join/Merge all of the dataframes together to display one global table that shows the three different receiver’s yards and touchdown (TD) data as well as every game result. Be sure to include Opponent, Location, and Result fields from the bengals.csv file.
Make any changes necessary to successfully merge/join the dataframes.

In [52]:
def join_player_stats(team_df: pd.DataFrame, player_df: pd.DataFrame, player_name: str) -> pd.DataFrame:
    return team_df.join(player_df.set_index('Week').add_prefix(player_name + ' '), on = 'Week', how = 'left')

In [85]:
joined_df = join_player_stats(bengals_df, boyd_df, 'Boyd')  # add Boyd stats to DataFrame
joined_df = join_player_stats(joined_df, chase_df, 'Chase')  # add Chase stats to DataFrame
joined_df = join_player_stats(joined_df, higgins_df, 'Higgins')  # add Higgins stats to DataFrame

 ### Replace the `1.0` or `0.0` values in the Result field to display `Win` or `Loss`, respectively.

In [83]:
def result_int_to_str(result: np.float64) -> str:
    if result == 1.0:
        return 'Win'
    elif result == 0.0:
        return 'Loss'
    else:
        return None

In [86]:
joined_df.loc[:, 'Result'] = [result_int_to_str(r) for r in joined_df['Result']]

### Write the dataframe to a postgresql DB

In [80]:
import json
from sqlalchemy import create_engine

# read database connection info from file
def get_db_url():
    with open('database.json', 'r') as f:
        config = json.load(f)
    db = config['database']
    return f'postgresql://{db["user"]}:{db["password"]}@{db["host"]}:{db["port"]}/{db["dbname"]}'

def get_database_connection():
    db = create_engine(get_db_url())
    return db.connect()

In [87]:
with get_database_connection() as conn:
    joined_df.to_sql('ryan_held', con = conn, if_exists = 'replace', index = False)